In [ ]:
%load_ext autoreload

In [ ]:
import os, sys
import holoviews as hv
sys.path.append(os.path.abspath('../../two_dim_majoranas/'))

import adaptive
adaptive.notebook_extension()

import numpy as np
import scipy.constants
import cmath
import kwant 

import functools as ft

import sns_system, plotting_results
from distributed_sns import AggregatesSimulationSet as ASS
from distributed_sns import SimulationSet as SS

# Define system

In [ ]:
constants = dict(
    m_eff=0.023 * scipy.constants.m_e / (scipy.constants.eV * 1e-3) / 1e18,  # effective mass in kg, 
    hbar=scipy.constants.hbar / (scipy.constants.eV * 1e-3),
    e = scipy.constants.e,
    current_unit=scipy.constants.k * scipy.constants.e / scipy.constants.hbar * 1e9,  # to get nA
    mu_B=scipy.constants.physical_constants['Bohr magneton'][0] / (scipy.constants.eV * 1e-3),
    k=scipy.constants.k / (scipy.constants.eV * 1e-3),
    exp=cmath.exp,
    cos=cmath.cos,
    sin=cmath.sin
   )

syst_pars = {'Ll' :  1000,
               'Lr' : 1000,
               'Lm' : 12.5,
               'Ly' : 12.5,
               'a' :  12.5,
            'mu_from_bottom_of_spin_orbit_bands': True}

params_raw= dict(g_factor_middle = 10,
                 g_factor_left = 0,
                 g_factor_right = 0,
                 mu = 1.0,
                 alpha_middle = 28,
                 alpha_left = 28,
                 alpha_right = 28,
                 Delta_left = .18,
                 Delta_right = .18,
                 B = 0.5,
                 phase = np.pi/2,
                 T = 0.025)

params = dict(**constants,
              **params_raw)

In [ ]:
locals().update(syst_pars)

In [ ]:
transverse_soi = True
tau_z = kwant.continuum.discretizer.ta.array([[1,0,0,0],
                                             [0,1,0,0],
                                             [0,0,-1,0],
                                             [0,0,0,-1]])
template_strings = sns_system.get_template_strings(transverse_soi, syst_pars['mu_from_bottom_of_spin_orbit_bands'])

# TURN HAMILTONIAN STRINGS INTO TEMPLATES
template_normal = kwant.continuum.discretize(template_strings['ham_normal'], grid_spacing=a)
template_sc_left = kwant.continuum.discretize(template_strings['ham_sc_left'], grid_spacing=a)
template_sc_right = kwant.continuum.discretize(template_strings['ham_sc_right'], grid_spacing=a)


def shape_normal(site):
    (x, y) = site.pos
    return 0 <= x < a and 0 <= y < a

def shape_left_normal(site):
    (x, y) = site.pos
    return y == 0

def shape_right_sc(site):
    (x, y) = site.pos
    return y == 0

# BUILD FINITE SYSTEM
sym = kwant.TranslationalSymmetry((0, a))#wraparound symmetry

sym_left = kwant.TranslationalSymmetry((-a,0), (0, a))
left_lead = kwant.Builder(sym_left, conservation_law=-tau_z,)
left_lead.fill(template_normal, shape_normal, (0,0))

sym_right = kwant.TranslationalSymmetry((a,0), (0, a))
right_lead = kwant.Builder(sym_right)
right_lead.fill(template_sc_right, shape_right_sc, (a,0))

middle = kwant.Builder(symmetry=sym, conservation_law=-tau_z)
middle.fill(template_normal, shape_normal, (0,0))

middle = kwant.wraparound.wraparound(middle, coordinate_names='y')
left_lead = kwant.wraparound.wraparound(left_lead, keep=0)
right_lead = kwant.wraparound.wraparound(right_lead, keep=0)

middle.attach_lead(left_lead)
middle.attach_lead(right_lead)
syst = middle.finalized()

In [ ]:
_=kwant.plot(syst)

In [ ]:
def transmission(e):
    def transmission_per_ky(k_y):
        _params = params.copy()
        _params['k_y'] = k_y
        smatrix = kwant.smatrix(syst, energy=e, params=_params)
        return smatrix.transmission((0,1),(0,1))

    learner = adaptive.IntegratorLearner(transmission_per_ky, [-np.pi, np.pi], tol=1e-1)
    runner = adaptive.runner.simple(learner, lambda l: l.done())
    return learner.igral
    

In [ ]:
learner = adaptive.Learner1D(transmission, [-np.pi/2, np.pi/2])

In [ ]:
runner = adaptive.Runner(learner)
runner.live_info()

In [ ]:
learner.plot()